In [1]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone 
import pinecone 
from langchain.document_loaders import PyPDFLoader , DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import CTransformers
from langchain import prompts

c:\Users\anant\anaconda3\envs\medicalchat\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

# Get the API key from the environment variable
KEY = os.getenv("PINECONE_API_KEY")



In [3]:
def load_pdf(data):
    loader = DirectoryLoader(data , glob="*.pdf" , loader_cls= PyPDFLoader)
    document = loader.load()
    return document

In [4]:
extracted = load_pdf("data/")

In [5]:
def text_split(extracted):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500 , chunk_overlap = 20)
    text_chunk = text_splitter.split_documents(extracted)

    return text_chunk

In [6]:
test_chunks = text_split(extracted)

In [7]:
len(test_chunks)

7020

In [8]:
#embedding 
def download_hugging_face_embedding():
    embeddings = HuggingFaceEmbeddings(model_name= "sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [9]:
embeddings = download_hugging_face_embedding()

c:\Users\anant\anaconda3\envs\medicalchat\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [10]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [11]:
from langchain.vectorstores import Pinecone as PineconeStore

In [12]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec


pc = Pinecone(api_key=KEY)


In [13]:
index_name = "medicalchat"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws', 
            region='us-east-1'
        ) 
    ) 


In [14]:
docsearch = PineconeStore.from_texts(
    [t.page_content for t in test_chunks],
    embeddings,
    index_name=index_name
)

In [15]:
docsearch = PineconeStore.from_existing_index(index_name , embeddings)

query = "what are allergies"

docs = docsearch.similarity_search(query , k = 2)
print(docs)

[Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE"), Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain h

In [16]:
prompt = """
Use the following piece of information to answer the user's question 
if you don't know the answer, say just that you don't know, don't try to make up an answer.

Contest :{context}
Question: {question}
Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [17]:
prompt1 = PromptTemplate(template=prompt , input_variables=["context" , "question"])
chain_type_kwargs = {"prompt":prompt1}

AttributeError: No huggingface_hub attribute HfHub

In [22]:
from langchain_community.llms import CTransformers
llm = CTransformers(
        model = "C:/Users/anant/deep learning/Medical_chat_bot/model/llama-2-7b-chat.ggmlv3.q4_0.bin",
        model_type="llama",
        config={'max_new_tokens':512,'temperature':0.8}
    )


In [23]:
qa = RetrievalQA.from_chain_type(
    llm= llm,
    chain_type = "stuff",
    retriever = docsearch.as_retriever(search_kwargs ={'k' : 2}),
    return_source_documents = True,
    chain_type_kwargs = chain_type_kwargs
)